In [ ]:
date = "20250519"

In [ ]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
import os
import gc


from sklearn.model_selection import train_test_split,KFold

from sklearn.metrics import confusion_matrix,f1_score,classification_report,accuracy_score
from sklearn.metrics import roc_auc_score
 
from scipy.stats import norm
import matplotlib.pyplot as plt 
# import seaborn as sns
import time
import os
import openpyxl

import joblib

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout,Flatten,BatchNormalization
from tensorflow.keras.layers import Conv1D,Input, Conv1DTranspose, Concatenate
from tensorflow.keras.layers import MaxPooling1D,UpSampling1D,Lambda,Reshape
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.models import Model,load_model

from tensorflow.keras.losses import mse,binary_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as BK

from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.optimizers import Adam

from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.utils import class_weight

# for now()
import datetime
# for timezone()
import pytz
# using now() to get current time
current_time = datetime.datetime.now(pytz.timezone('America/New_York'))
# printing current time in india
print("The current time is :", current_time)

pd.options.mode.chained_assignment = None

np.seterr(divide='ignore', invalid='ignore')

In [ ]:
# _celline = ['A549','GM12878','H1.hESC','HeLa.S3','HepG2','HT1080','HUVEC','IMR.90','K562','MCF.7','NCI.H460','NHEK','SK.MEL.5','SK.N.DZ','SK.N.SH']
_celline = ['IMR.90','K562','HepG2']
K = 6
miss = [2]

threshold = "0"

encoding_dim = 512

# # input path
path_input = "../data/mer%s/"%(K)
# output path
path_output = "../results/Train_one_test_all/with_AE/mer%s_%s/"%(K,date)

path_models = path_output+"models/"
ae_path = path_output + "ae_models/"

os.makedirs(path_output, exist_ok=True)
os.makedirs(path_models,exist_ok=True)
os.makedirs(ae_path, exist_ok=True)

CV = 5
max_len = 5000
segment = [(200,5000)]

_model_type = ["1DCNN"]
ae_type = ["AE_MLP"]

encoding = "rawcounts"

# <font color=blue> Load data</font>
##  <font color=blue> load cell_line, kmer counts table, generate two class training set, mean as threshold. </font>

In [ ]:
def assign_class(row, Celline):
    if row[Celline] < 0:
        return 0
    else:
        return 1


def processdata(df):
    x_train_ = df.iloc[:,0:-5]
    y_train_ = df.iloc[:,-1]
    train_rci_ = df.iloc[:,-5:]
    
    return x_train_,y_train_,train_rci_

from collections import defaultdict
from math import sqrt
def evaluate_matrix(cm):
    idx = [0,1]
    columns = ['Sn','Sp','Preci','MCC','F1-score','OA']
    eva = pd.DataFrame(index=idx,columns=columns)
    
    oa = np.diag(cm).sum()/cm.sum()*100
    
    for i in range(len(cm)):
        fp = cm.sum(axis=0)[i]-cm[i][i] # sum of column
        fn = cm.sum(axis=1)[i]-cm[i][i] # sum of row
        tp = cm[i][i]
        tn = cm.sum()-(fp+fn+tp)
        if tp == 0:
            sensitivity = 0
            precision = 0
        else:
            sensitivity = tp / (tp+fn)
            precision = tp / (tp+fp)
            
        specificity = tn / (tn+fp)
               
        if precision*sensitivity==0:
            f1_score=0
        else:
            f1_score = 2 * precision * sensitivity / (precision + sensitivity)
        if tp*tn-fp*fn == 0:
            mcc = 0
        else:
            mcc = (tp*tn-fp*fn)/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
        if i==0:
            eva.loc[i] = [sensitivity,specificity,precision,mcc,f1_score,oa]
        else:
            eva.loc[i] = [sensitivity,specificity,precision,mcc,f1_score,0]
    
    return oa , eva

def build_CNN(input_shape):
    model = Sequential([
        Conv1D(filters=128, kernel_size=3, activation='relu', kernel_initializer=HeNormal(), input_shape=(input_shape, 1)),
        MaxPooling1D(pool_size=2),
        
        Conv1D(filters=256, kernel_size=3, activation='relu', kernel_initializer=HeNormal()),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(256, activation='relu', kernel_initializer=HeNormal()),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
#     cnn1.summary()

    return model

def trainModel(ae_type,n_fold,m,n,model,celline,model_types,x_train,x_val,y_train, y_val):

    early_stopping = EarlyStopping(monitor='val_loss', mode="min",patience=10,verbose=0)
    model_checkpoint = ModelCheckpoint(path_models+'%s_%s_%s_%s_%s_%smer%smiss_%s_fold%s.h5'%(encoding,celline,model_types,m,n,K,Q,ae_type,n_fold), monitor='val_loss',
                                       mode='min', verbose=0, save_best_only=True,save_weights_only=True)

    # fit the keras model on the dataset
    history = model.fit(x_train, y_train, epochs=30,
                        batch_size=128,
                        verbose=0,
                        callbacks=[early_stopping,model_checkpoint],
                        validation_data=(x_val, y_val))
    # evaluate the keras model
    model.load_weights(path_models+'%s_%s_%s_%s_%s_%smer%smiss_%s_fold%s.h5'%(encoding,celline,model_types,m,n,K,Q,ae_type,n_fold))
    score = model.evaluate(x_val, y_val)
    print("val accuracy: ",score[1]*100)

    return model,score[1]

def evaluateModel(model_types,model,x_val,y_val,dff_val):
    if model_types == "RF":
        y_pred_prob = model.predict_proba(x_val)[:, 1] 
    else:
        y_pred_prob = model.predict(x_val).ravel() 
#     print(f'predict prob: {y_pred_prob}')
    auc = roc_auc_score(y_val, y_pred_prob)
    
    y_pred = (model.predict(x_val) > 0.5).astype("int32")    
    accuracy = accuracy_score(y_val, y_pred)
    
    cm = confusion_matrix(y_val, y_pred)
    oa, eva = evaluate_matrix(cm)
    eva.loc[0,"AUC"] = auc
    dff_val = pd.concat([dff_val,pd.DataFrame(y_pred,columns=["pred"]),pd.DataFrame(y_pred_prob,columns=["prob"])],axis=1)

    return oa, cm, eva,dff_val,auc 

In [ ]:
########################################
def AE_mlp(x_train_,x_val_,input_dim):
    input_layer = Input(shape=(input_dim,))
    # Encoder
    encoded = Dense(256, activation='relu')(input_layer)
    encoded = BatchNormalization()(encoded)
    encoded = Dense(128, activation='relu')(encoded)
    encoded = BatchNormalization()(encoded)
    encoded = Dropout(0.3)(encoded)
    encoded = Dense(encoding_dim, activation='relu')(encoded)

    # Decoder
    decoded = Dense(128, activation='relu')(encoded)
    decoded = Dense(256, activation='relu')(decoded)
    decoded = Dense(input_dim, activation="sigmoid")(decoded)
    # define autoencoder model
    ae_mlp = Model(input_layer, decoded)
    # compile autoencoder model
    
    ae_mlp.compile(optimizer='adam', loss='binary_crossentropy')
    ae_mlp.summary()

#     checkpointer = ModelCheckpoint(filepath=path1+'bestmodel.h5', verbose=0, save_best_only=True)
    earlystopper = EarlyStopping(monitor='val_loss', patience=10, verbose=0, restore_best_weights=True)

    #v fit the autoencoder model to reconstruct input
    history = ae_mlp.fit(x_train_, x_train_,
                        epochs=200, batch_size=128, verbose=0,
                        callbacks=[earlystopper],
                        validation_data=(x_val_,x_val_))
    
    encoder_mlp = Model(input_layer,encoded, name='ae_mlp')
    # plot loss
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='val')
    plt.legend()
    plt.show()

    return encoder_mlp

In [ ]:
def train_model(Cell_train):
    
    # COUNT Running time
    start = time.time()

    for j in range(0,2):
   
        # training dataset split by genes, 5-fold CV
        kf = KFold(n_splits=CV,random_state=(j+1)*42, shuffle=True)
        i=0
        Score = {"1DCNN":0,"MLP":0} 
        for train_index, val_index in kf.split(genes):
            i += 1
            print("No.{} fold".format(j*5+i))      
            for (m,n) in segment:

                df_train = _df_train.copy()   ## here

                train = df_train.iloc[train_index]
                train = train.reset_index(drop=True)                
                x_train,y_train,train_rci = processdata(train)               
            
                count = Counter(y_train)
                ratio = count[0]/count[1]
                print("class 0 : class 1 = %.3f."%(ratio))

                val = df_train.iloc[val_index].reset_index(drop=True)
                x_val,y_val,val_rci = processdata(val)
    
                x_train = x_train.to_numpy()
                x_val = x_val.to_numpy()
  
                input_dim = x_train.shape[1]

                for ae in ae_type:
                    smt = SMOTE(sampling_strategy='minority', random_state=42)
                    if ae == "AE_MLP":
                        enc_mlp = AE_mlp(x_train,x_val,input_dim)
                        enc_mlp.save(ae_path+"%s_Celline_%s_fea%s_latent%s_%smer%smiss_fold%s.h5"%(ae,Cell_train,input_dim,encoding_dim,K,Q,j*5+i))
#                         enc_mlp = load_model(ae_path+"%s_Celline_%s_fea%s_latent%s_%smer%smiss_fold%s.h5"%(ae,Celline,input_dim,encoding_dim,K,Q,j*5+i),compile=False)
                        x_train_ae = enc_mlp.predict(x_train)
                        x_train_ae, y_train_ae = smt.fit_resample(x_train_ae, y_train)
                        print("%s After over-sampling %s"%(ae,Counter(y_train_ae)))
                        x_val_ae = enc_mlp.predict(x_val)

                    for model_type in _model_type:
                        print()
                        ### load model
                        print("\nTraining on %s %smer%smismatch %s,%s %s %s %s. "%(Cell_train,K,Q,ae,encoding, model_type,m,n))
                        print()
                        if model_type == "1DCNN":
                            k = x_train_ae.shape[1]
                            model = build_CNN(x_train_ae.shape[1])
                            x_train_cnn = x_train_ae.reshape(len(x_train_ae),k,1)
                            x_val_cnn = x_val_ae.reshape(len(x_val_ae),k,1)
                            model,score = trainModel(ae,j*5+i,m,n,model,Cell_train,model_type,x_train_cnn,x_val_cnn,y_train_ae, y_val)
                            oa_val, cm_val, eva_val, val0_predict,auc_val = evaluateModel(model_type,model,x_val_cnn,y_val,val_rci)
                        print(f'model {model_type} val accu {oa_val:.3f}, auc {auc_val:.3f}\nval cm {cm_val}')
                        eva_val0_file =path_output + "10fold_train_%s_val_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_val0.xlsx"%(Cell_train,Cell_train,m,n,encoding,threshold,model_type,K,Q)
                        predict_file_val = path_output + "10fold_train_%s_val_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_predict_val.xlsx"%(Cell_train,Cell_train,m,n,encoding,threshold,model_type,K,Q)

                        writer_eva_val = ExcelWriter(eva_val0_file, engine='openpyxl',mode='a')
                        writer_predict_val = ExcelWriter(predict_file_val, engine='openpyxl',mode='a')

                        conf_matrix_df = pd.DataFrame(cm_val, columns=['Pred_0', 'Pred_1'], index=['True_0', 'True_1'])    
                        eva_val.to_excel(writer_eva_val,sheet_name="eva{}".format(j*5+i))
                        conf_matrix_df.to_excel(writer_eva_val,sheet_name="CM{}".format(j*5+i))                    
                        val0_predict.to_excel(writer_predict_val,sheet_name="predict_0_{}".format(j*5+i))
                        writer_eva_val.close()
                        writer_predict_val.close()      

                        
                        for Cell_test in _celline:
                           
                            print("\nfold %s, train %s, test %s"%(j*5+i,Cell_train,Cell_test))
                            _df_test = data_test.copy()
                            _df_test[Cell_test] = _test[Cell_test]
                            _df_test = _df_test.dropna(axis=0).reset_index(drop=True)
                            _df_test["Class"] = _df_test.apply(lambda row:assign_class(row,Cell_test),axis=1)

                            x_test,y_test,test_rci = processdata(_df_test)
                            x_test = x_test.to_numpy()
                            x_test_ae = enc_mlp.predict(x_test)
                            x_test = x_test_ae.reshape(len(x_test_ae),k,1)
                            oa_test, cm_test,eva_test,test0_predict,auc_test = evaluateModel(model_type,model,x_test,y_test,test_rci)
                            print(f'model {model_type} test accu {oa_test:.3f}, auc {auc_test:.3f}\ntest cm {cm_test}')    

                            ## prediction on test,threshold 0 and mean+/-std


                            eva_test0_file = path_output + "10fold_train_%s_test_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_test0.xlsx"%(Cell_train,Cell_test,m,n,encoding,threshold,model_type,K,Q)
                            predict_file_test = path_output + "10fold_train_%s_test_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_predict_test.xlsx"%(Cell_train,Cell_test,m,n,encoding,threshold,model_type,K,Q)

                            writer_eva_test = ExcelWriter(eva_test0_file, engine='openpyxl',mode='a')
                            writer_predict_test = ExcelWriter(predict_file_test, engine='openpyxl',mode='a') 

                            conf_matrix_df = pd.DataFrame(cm_test, columns=['Pred_0', 'Pred_1'], index=['True_0', 'True_1'])
                            eva_test.to_excel(writer_eva_test,sheet_name="eva{}".format(j*5+i))
                            conf_matrix_df.to_excel(writer_eva_test,sheet_name="CM{}".format(j*5+i))
                            test0_predict.to_excel(writer_predict_test,sheet_name="predict_0_{}".format(j*5+i))                  

                            writer_eva_test.close()
                            writer_predict_test.close()

                        tf.keras.backend.clear_session()
                        gc.collect()
    end = time.time()
    print("Running time {} seconds".format(round(end-start,3)))

## <font color="blue"> only validation </font>

In [ ]:
for Q in miss:
    _train = pd.read_csv(path_input+encoding+"_noncoding_train_%smer%smiss.csv"%(K,Q),index_col=0)
    data_train = _train.iloc[:,0:-18].div(_train.iloc[:,0:-18].sum(axis=1), axis=0)
    data_train[["transcript_id","gene_id","length"]] = _train[["transcript_id","gene_id","length"]]

    _test = pd.read_csv(path_input+encoding+"_noncoding_test_%smer%smiss.csv"%(K,Q),index_col=0)
    data_test = _test.iloc[:,0:-18].div(_test.iloc[:,0:-18].sum(axis=1), axis=0)
    data_test[["transcript_id","gene_id","length"]] = _test[["transcript_id","gene_id","length"]]



    #Create output files
    wb = openpyxl.Workbook()

    for Cell_train in _celline:

        for Cell_test in _celline:
            for model_type in _model_type:

                for (m,n) in segment:
                    
                    eva_val0_file = path_output + "10fold_train_%s_val_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_val0.xlsx"%(Cell_train,Cell_test,m,n,encoding,threshold,model_type,K,Q)
                    predict_file_val = path_output + "10fold_train_%s_val_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_predict_val.xlsx"%(Cell_train,Cell_test,m,n,encoding,threshold,model_type,K,Q)
                    eva_test0_file = path_output + "10fold_train_%s_test_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_test0.xlsx"%(Cell_train,Cell_test,m,n,encoding,threshold,model_type,K,Q)
                    predict_file_test = path_output + "10fold_train_%s_test_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_predict_test.xlsx"%(Cell_train,Cell_test,m,n,encoding,threshold,model_type,K,Q)
                    wb.save(eva_val0_file)                                         
                    wb.save(predict_file_val)
                    wb.save(eva_test0_file)
                    wb.save(predict_file_test)

    ## Whole training mean and std as test threshold
    for Cell_train in _celline:
        _df_train = data_train.copy()
        _df_train[Cell_train] = _train[Cell_train]
        _df_train = _df_train.dropna(axis=0)
        _df_train = _df_train.sort_values(by="length",ascending=False).reset_index(drop=True)
        _df_train["Class"] = _df_train.apply(lambda row:assign_class(row,Cell_train),axis=1)


        genes = pd.DataFrame(_df_train.gene_id.unique(),columns=["gene_id"])

        print()
        print("\n training on %s %smer%smismatch ."%(Cell_train,K,Q ))

        train_model(Cell_train)

In [ ]:
for Q in miss:
    for model_type in _model_type:
        for (m,n) in segment:
            for Cell_train in _celline:
                for Cell_test in _celline:
                    for ae in ae_type:
#                         eva_val0_file = path1 + "10fold_train_%s_test_%s_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_val0.xlsx"%(Cell_train,Cell_test,ae,m,n,encoding,threshold,model_type,K,Q)
                        eva_test0_file = path_output + "10fold_train_%s_test_%s_training_length_%s_%s_%s_%s_%s_%smer%smiss_test0.xlsx"%(Cell_train,Cell_test,m,n,encoding,threshold,model_type,K,Q)
                        for infile in [eva_test0_file]:
                            print(infile)
                            writer = ExcelWriter(infile, engine='openpyxl',mode='a',if_sheet_exists='replace')
                            xl = pd.ExcelFile(infile)
                            eva = []

                            for i in range(1,11):
                                eva.append(xl.parse('eva'+str(i),index_col=0))

                            df_eva_avg =  pd.concat(eva).groupby(level=0).mean()

                            df_eva_avg.to_excel(writer,sheet_name='eva_avg')

                            writer.close()

In [ ]:
current_time = datetime.datetime.now(pytz.timezone('America/New_York'))
# printing current time in india
print("The current time is :", current_time)